In [14]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

In [7]:
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
data_types = ['Wake','N1', 'N2', 'N3', 'REM']
loaded_data = {}


for data_type in data_types:
    print(data_type)
    X_y_groups = {}
    
    X = pd.read_hdf(load_path + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
    X, y , groups = X.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

    #Transform the X into TS_Fresh Features___
    # 1. Convert to TS_Fresh format Dataframe 
    ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
    
    # 2. Extract TS_Fresh Features from the dataframe
    settings = MinimalFCParameters()
    extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)

    # 3. Asign extract_ts_fresh_df to the variable X
    X = extracted_ts_fresh_df.copy()
        
    #Drop columns where all values are NA 
    X = X.dropna(axis = 1)
    
    #Drop columns where all values are the same
    # Find columns where all values are the same
    same_value_columns = X.columns[X.nunique() == 1]
    # Drop columns with the same values
    X = X.drop(columns=same_value_columns)

    #Generate regional features dict per data type 
    #Generate a region to features dictionary - this will enable us to run expts regionally as before
    regional_features_dict = {}
    region_channel_dict = constants.region_to_channel_dict
    regions = list(region_channel_dict.keys())
    for region in regions:
        region_features = [col for col in X.columns if region + '_' in col]
        if len(region_features) > 0 : 
            regional_features_dict[region] = region_features
       
       

    X_y_groups['X'] = X
    X_y_groups['y'] = y
    X_y_groups['groups'] = groups
    X_y_groups['regions_dict'] = regional_features_dict

    loaded_data[data_type] = X_y_groups

Wake


14-Sep-23 11:22:18 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:01<00:00, 72.83it/s]


N1


14-Sep-23 11:22:21 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 59.07it/s]


N2


14-Sep-23 11:22:25 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 48.07it/s]


N3


14-Sep-23 11:22:30 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:02<00:00, 52.63it/s]


REM


14-Sep-23 11:22:34 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 51.90it/s]


### Expt_1

In [19]:
#Run Expt 1 experiments 

t1 = time.time()

expt_1_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_1_results[data_type] = results_df

t2 = time.time()

t2-t1

109.21174097061157

In [20]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_1_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
    sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
    sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))

    df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
    df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
    df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
    
    display(df)

Wake


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.750000,0.750000,0.516667,0.566667,0.566667,0.566667,0.516667,0.533333,0.483333,0.433333,0.583333,0.616667,0.600000,0.550000
0,DT,0.650000,0.633333,0.583333,0.483333,0.600000,0.500000,0.633333,0.533333,0.633333,0.533333,0.516667,0.483333,0.650000,0.500000
0,Ada_B,0.666667,0.666667,0.566667,0.600000,0.566667,0.533333,0.550000,0.616667,0.566667,0.450000,0.550000,0.550000,0.666667,0.566667


N1


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.8250,0.7625,0.7625,0.7875,0.7125,0.8125,0.7875,0.7375,0.7625,0.7500,0.775,0.7250,0.8250,0.8125
0,DT,0.7875,0.7500,0.6250,0.5125,0.6125,0.6875,0.6500,0.6500,0.6625,0.7000,0.600,0.7000,0.7125,0.7875
0,Ada_B,0.7375,0.6250,0.6125,0.6125,0.5750,0.6500,0.6375,0.6875,0.6250,0.6125,0.700,0.7375,0.6250,0.6625


N2


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.595833,0.483333,0.461111,0.440278,0.484722,0.502778,0.437500,0.502778,0.450000,0.595833,0.420833,0.397222,0.552778,0.469444
0,DT,0.605556,0.434722,0.344444,0.463889,0.536111,0.548611,0.481944,0.605556,0.461111,0.580556,0.408333,0.405556,0.430556,0.437500
0,Ada_B,0.584722,0.477778,0.511111,0.425000,0.486111,0.377778,0.534722,0.425000,0.584722,0.544444,0.480556,0.347222,0.480556,0.409722


N3


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.595833,0.511111,0.486111,0.519444,0.519444,0.595833,0.558333,0.559722,0.533333,0.545833,0.544444,0.520833,0.545833,0.509722
0,DT,0.683333,0.612500,0.523611,0.462500,0.461111,0.494444,0.522222,0.683333,0.593056,0.402778,0.630556,0.422222,0.520833,0.463889
0,Ada_B,0.631944,0.584722,0.363889,0.522222,0.498611,0.630556,0.509722,0.631944,0.531944,0.454167,0.558333,0.531944,0.508333,0.512500


REM


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.6375,0.5750,0.5000,0.5375,0.5125,0.6000,0.5125,0.550,0.525,0.5750,0.575,0.6375,0.4750,0.4375
0,DT,0.6375,0.5125,0.4500,0.5125,0.5500,0.6375,0.5625,0.600,0.575,0.5375,0.500,0.5625,0.3625,0.4875
0,Ada_B,0.6500,0.4750,0.5125,0.4750,0.5125,0.5875,0.5750,0.475,0.600,0.5125,0.525,0.6500,0.4375,0.4125


In [17]:
### Expt_2

In [21]:
#Run Expt 1 experiments 

t1 = time.time()

expt_2_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    regional_features_dict =  X_y_groups['regions_dict']
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_2_results[data_type] = results_df

t2 = time.time()

t2-t1

112.25525522232056

In [22]:
for data_type in data_types:
    print(data_type)
    results_df = expt_2_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
    sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
    sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))

    df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
    df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
    df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
    
    display(df)

Wake


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.737500,0.737500,0.626786,0.673214,0.575000,0.473214,0.432143,0.535714,0.435714,0.501786,0.573214,0.433929,0.562500,0.453571
0,DT,0.746429,0.746429,0.512500,0.494643,0.523214,0.432143,0.462500,0.539286,0.358929,0.514286,0.555357,0.489286,0.566071,0.569643
0,Ada_B,0.719643,0.719643,0.551786,0.657143,0.655357,0.594643,0.546429,0.628571,0.500000,0.498214,0.571429,0.460714,0.548214,0.542857


N1


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.767778,0.641111,0.513333,0.580000,0.492222,0.658889,0.630000,0.645556,0.626667,0.656667,0.670000,0.660000,0.670000,0.767778
0,DT,0.785556,0.615556,0.643333,0.552222,0.650000,0.617778,0.587778,0.500000,0.524444,0.534444,0.615556,0.660000,0.593333,0.785556
0,Ada_B,0.692222,0.588889,0.542222,0.515556,0.628889,0.466667,0.563333,0.573333,0.575556,0.543333,0.533333,0.617778,0.627778,0.692222


N2


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.626667,0.478889,0.454444,0.394444,0.456667,0.517778,0.480000,0.570000,0.545556,0.626667,0.531111,0.458889,0.577778,0.425556
0,DT,0.627778,0.542222,0.371111,0.483333,0.485556,0.496667,0.544444,0.584444,0.552222,0.573333,0.588889,0.490000,0.627778,0.342222
0,Ada_B,0.613333,0.553333,0.434444,0.497778,0.501111,0.532222,0.531111,0.507778,0.450000,0.613333,0.605556,0.477778,0.518889,0.396667


N3


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.553333,0.524444,0.528889,0.520000,0.498889,0.501111,0.512222,0.553333,0.538889,0.448889,0.520000,0.518889,0.553333,0.535556
0,DT,0.564444,0.524444,0.465556,0.436667,0.467778,0.492222,0.501111,0.564444,0.500000,0.447778,0.531111,0.513333,0.482222,0.545556
0,Ada_B,0.591111,0.491111,0.540000,0.541111,0.551111,0.481111,0.564444,0.541111,0.591111,0.448889,0.548889,0.426667,0.476667,0.481111


REM


,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.598889,0.520000,0.488889,0.531111,0.447778,0.598889,0.510000,0.513333,0.488889,0.593333,0.424444,0.476667,0.360000,0.445556
0,DT,0.555556,0.436667,0.551111,0.475556,0.488889,0.543333,0.545556,0.470000,0.502222,0.443333,0.437778,0.555556,0.347778,0.497778
0,Ada_B,0.585556,0.402222,0.410000,0.517778,0.488889,0.511111,0.554444,0.371111,0.525556,0.474444,0.446667,0.585556,0.426667,0.572222
